In [23]:
import torch
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
from numpy.testing import assert_array_equal, assert_array_almost_equal, assert_equal, assert_almost_equal
from pandas.testing import assert_frame_equal
from sklearn.tree import DecisionTreeRegressor as DTR, DecisionTreeClassifier as DTC
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression, make_classification
import matplotlib.pyplot as plt 
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingClassifier
from sklearn.metrics import mean_squared_error as MSE


# SLIDE (1) Bootstrap.

На вход массив чисел $X$ и число бутстрепных выборок $B$. Необходимо реализовать свой бутстреп и найти матожидание и стандартную ошибку у бутстрепных выборок.

### Sample 1
#### Input:
```python
X = np.array([37,43,38,36,17,40,40,45,41,84])
B = 100000
```
#### Output:
```python
42.1, 4.56
```


# TASK

In [28]:
import numpy as np
from scipy.stats import sem # ищет SE среднего

def get_stats(X: np.array, B:int)->tuple:
    SEs = []
    means = []
    for _ in range(B):
        sample = np.random.choice(X, len(X), True)
        SEs.append(sem(sample))
        means.append(sample.mean())
    mean = np.array(means).mean()
    SE = np.array(SEs).mean()
    return mean, SE

# OPEN TESTS

In [30]:
######################################################
X = np.array([37,43,38,36,17,40,40,45,41,84])
B = 10000

mean, se = get_stats(X, B)
print(mean)
print(se)

assert np.abs(mean - 42.1) < 0.05
assert np.abs(se - 4.56) < 0.03
######################################################
print('Well Done!')

42.13739
4.585872232799194
Well Done!


# SLIDE (1) Bias-variance

На вход подается **один** объект $(x, y)$ и список из нескольких **обученных** моделей. 

Необходимо найти $error$, $bias^2$, $variance$ для данного объекта.

Теперь все аккуратно запишем, чтобы не запутаться.

* $(x, y)$ - тестировачная выборка
* $a_1(\cdot), \ldots, a_M(\cdot)$ - модели (это не обученные на бутстрепе модели, а просто возможные модели из пространства $\mathbb{A}$, которое мы выбрали)

Как настоящие статистики мы можем ~~забить~~ оценить матожидание как среднее.**Это не смешанная модель, а именно оценка матожидания через среднее**
$$\mathbb{E}a(x) = \frac{1}{M}\sum_{i=1}^{M}a_i(x)$$

**Error** (берем матожидание от квадрата разности)

$$error = \mathbb{E}_{a}(a(x)-y)^2 = \frac{1}{M}\sum_{i=1}^{M}(a_i(x) - y)^2$$

**Bias** (заметьте, что возвращаем квадрат bias, а не просто bias)

$$bias^2 = \Big(y - \mathbb{E}_{a}[a(x)]\Big)^2 = \Big(y - \frac{1}{M}\sum_{i=1}^{M}a_i(x)\Big)^2$$  


**Variance** (ищем смещенную оценку)

$$variance = \mathbb{D}_{a}a(x)= \mathbb{E}_{a}(a(x) - \mathbb{E}_{a}a(x))^2 = \frac{1}{M}\sum_{i=1}^{M}\Big(a_i(x)-\frac{1}{M}\sum_{r=1}^{M}a_r(x)\Big)^2$$

### Sample 1
#### Input:
```python
x, y = np.array([[0,0,0]]), 0
estimators = [DecisionTreeRegressor(max_depth=3, random_state=1),  #already fitted estimators
              DecisionTreeRegressor(max_depth=5, random_state=1)]
```
#### Output:
```python
error, bias2, var = 3.574, 3.255, 0.319
```

# TASK

In [85]:
import numpy as np

def bias_variance_decomp(x_test:np.array, y_test:int, estimators:list)->tuple:
    error = 0
    bias2 = 0
    var = 0
    est_count = len(estimators)
    est_sum = 0
    for estimator in estimators:
        y_ = estimator.predict(x_test)
        est_sum += y_
    for estimator in estimators:
        y_ = estimator.predict(x_test)
        error += pow(y_test - y_, 2)
        var += pow(y_ - est_sum / est_count, 2)
    error /= est_count
    bias2 = pow(y_test - est_sum / est_count, 2)
    var = var / est_count
    return error[0], bias2[0], var[0]

# OPEN TESTS

In [86]:
def generate(n_samples, noise, f):
    X = np.linspace(-4, 4, n_samples)
    y = f(X)
    X = X.reshape((n_samples, 1))

    return X, y
######################################################

n_train = 150        
noise = 0.1

# Generate data
def f(x):
    x = x.ravel()
    return np.exp(-x ** 2) + 1.5 * np.exp(-(x - 2) ** 2)

X, y = generate(n_samples=n_train, noise=noise, f=f)

estimators = [DTR(max_depth=2, random_state=1).fit(X, y), 
              DTR(max_depth=4, random_state=1).fit(X, y)]

x, y = np.array([[2]]), 1.5

error, bias, var = bias_variance_decomp(x, y, estimators)

assert_array_almost_equal(np.array([error, bias, var]), 
                          np.array([0.108, 0.083, 0.025]), decimal=3)

x, y = np.array([[-0.7]]), 0.8
error, bias, var = bias_variance_decomp(x, y, estimators)

assert_array_almost_equal(np.array([error, bias, var]), 
                          np.array([0.045, 0.002, 0.043]), decimal=3)

######################################################

X, y = make_regression(n_samples=1000, n_features=3, n_informative=3, bias=2, noise=10, 
                       n_targets=1, shuffle=False, random_state=10)

estimators = [DTR(max_depth=3, random_state=1).fit(X, y), 
              DTR(max_depth=5, random_state=1).fit(X, y)]

x, y = np.array([[0,0,0]]), 0
error, bias, var = bias_variance_decomp(x, y, estimators)

assert_array_almost_equal(np.array([error, bias, var]), 
                          np.array([3.574, 3.255, 0.319]), decimal=3)

print('Well Done')

Well Done


# SLIDE (1) Bias-variance v2

А теперь тоже самое, только для нескольких объектов

На вход подается тестовая выборка объект $(X_test, y_test)$ и список из нескольких **обученных** моделей. 

Необходимо найти $error$, $bias^2$, $variance$, $noise$ для данного объекта.

$$error = \mathbb{E}_{x,y}\mathbb{E}_{a}(a(x)-y)^2 = \frac{1}{N}\sum_{i=1}^{N}\frac{1}{M}\sum_{j=1}^{M}(a_j(x_i) - y_i)^2$$

$$bias^2 = \mathbb{E}_{x,y}\Big(y - \mathbb{E}_{a}[a(x)]\Big)^2 = \frac{1}{N}\sum_{i=1}^{N}\Big(y_i - \frac{1}{M}\sum_{j=1}^{M}a_j(x_i)\Big)^2$$  

$$variance = \mathbb{E}_{x,y}\mathbb{D}_{a}a(x)= \mathbb{E}_{x,y}\mathbb{E}_{a}(a(x) - \mathbb{E}_{a}a(x))^2 = \frac{1}{N}\sum_{i=1}^{N}\frac{1}{M}\sum_{j=1}^{M}\Big(a_j(x_i)-\frac{1}{M}\sum_{r=1}^{M}a_r(x_i)\Big)^2$$


### Sample 1
#### Input:
```python
x = np.array([[  0,   0,   0],
              [0.1, 0.1, 0.1]])
y = np.array([0, 0.1])

estimators = [DecisionTreeRegressor(max_depth=3, random_state=3), 
              DecisionTreeRegressor(max_depth=5, random_state=3)]
```
#### Output:
```python
error, bias2, var = 3.399, 3.079, 0.319
```

# TASK

In [5]:
import numpy as np

def bias_variance_decomp2(x_test:np.array, y_test:np.array, estimators:list)->tuple:
    error = 0
    bias2 = 0
    var = 0
    obj_count = x_test.shape[0]
    est_count = len(estimators)
    est_mean = 0
    for i in range(obj_count):
        x = x_test[i].reshape(1, -1)
        y = y_test[i]
        bias = 0
        for estimator in estimators:
            prediction = estimator.predict(x)
            error += pow(prediction - y, 2)
            bias += prediction
            est_mean = 0
            for est in estimators:
                est_mean += est.predict(x)
            est_mean /= est_count
            var += pow(prediction - est_mean, 2)
        bias /= est_count
        bias = pow(y - bias, 2)
        bias2 += bias
    error /= est_count
    error /= obj_count
    bias2 /= obj_count
    var /= est_count
    var /= obj_count
    return error[0], bias2[0], var[0]

# OPEN TESTS

In [6]:
def generate(n_samples, noise, f):
    X = np.linspace(-4, 4, n_samples)
    y = f(X)
    X = X.reshape((n_samples, 1))

    return X, y
######################################################

n_train = 150        
noise = 0.1

# Generate data
def f(x):
    x = x.ravel()
    return np.exp(-x ** 2) + 1.5 * np.exp(-(x - 2) ** 2)


X, y = generate(n_samples=n_train, noise=noise, f=f)

estimators = [DTR(max_depth=2, random_state=1).fit(X, y), 
              DTR(max_depth=4, random_state=1).fit(X, y)]

x = np.array([[2], [-0.7]]) 
y = np.array([1.5, 0.8])

error, bias, var = bias_variance_decomp2(x, y, estimators)

assert_array_almost_equal(np.array([error, bias, var]), 
                          (np.array([0.108, 0.083, 0.025]) + np.array([0.045, 0.002, 0.043])) / 2, decimal=3)

######################################################

X, y = make_regression(n_samples=1000, n_features=3, n_informative=3, bias=2, noise=10, 
                       n_targets=1, shuffle=False, random_state=10)

estimators = [DTR(max_depth=3, random_state=1).fit(X, y), 
              DTR(max_depth=5, random_state=1).fit(X, y)]

x = np.array([[  0,   0,   0]])
y = np.array([0])

error, bias, var = bias_variance_decomp2(x, y, estimators)

assert_array_almost_equal(np.array([error, bias, var]), 
                          np.array([3.574, 3.255, 0.319]), decimal=3)


x = np.array([[  0,   0,   0],
              [0.1, 0.1, 0.1]])
y = np.array([0, 0.1])

error, bias, var = bias_variance_decomp2(x, y, estimators)

assert_array_almost_equal(np.array([error, bias, var]), 
                          np.array([3.399, 3.079, 0.319]), decimal=3)

print('Well Done')

[[2.]]
[[-0.7]]
[[0 0 0]]
[[0. 0. 0.]]
[[0.1 0.1 0.1]]
Well Done


# SLIDE (2) Bagging

На вход подается некий **необученный** алгоритм регрессии, тренировачная и тестовые выборки и число бутстрепных выборок. Необходимо 
* бустингом сделать несколько выборок $X_1, \ldots, X_B$
* обучить несколько алгоритмов на этих выборках: $a_1(\cdot), \ldots, a_B(\cdot)$
* реализовать бэггинг этого алгоритма и найти собственно предсказания, $error$, $bias^2$ и $variance$.

Вот теперь аккуратно. Это - **не матожидание**! Это модель такая.
$$a(x) = \frac{1}{B}\sum_{b=1}^{B}a_b(x)$$

А вот ее матожидание равно для всех алгоритмов:
$$\mathbb{E}_aa(x) = \mathbb{E}_a\frac{1}{B}\sum_{b=1}^{B}a_b(x) = \mathbb{E}_aa_1(x)$$

Но так как теперь, нам нужно посчитать матожидание, мы воспользуемся нашим множеством алгоритмов, обученных на бутстрепе, чтобы получить оценку матожидания единичного алгоритма.

$$\mathbb{E}_aa_1(x) = \frac{1}{B}\sum_{j=1}^{B}a_j(x)$$

Остальные формулы берутся из предыдущей задачи.

P.S.
* Так как тут есть вероятности, в целом тесты могут `редко` не взлететь. Перезашлите задачу в этом случае.

### Sample 1
#### Input:
```python
estimator = DecisionTreeRegressor(max_depth=1)
X_train = np.array([[1, 1], [2, 2]])
y_train = np.array([1, 2])
X_test  = np.array([[0, 0], [4, 4], [8, 8]])
y_test  = np.array([0, 4, 8])

B = 10
```
#### Output:
```python
y_pred = np.array([3.708, 6.016])
error  = 3.5 
bias^2 = 0.1
var    = 3.5
```

# TASK

In [64]:
import numpy as np
from sklearn.base import clone
from sklearn.tree import DecisionTreeRegressor as DTR
from numpy.testing import assert_array_equal, assert_array_almost_equal, assert_equal, assert_almost_equal
from pandas.testing import assert_frame_equal

def bagging(estimator, X_train, y_train, X_test, y_test, boot_count):
    obj_count = X_train.shape[0]
    X_samples, y_samples = get_samples(X_train, y_train, boot_count)
    
    estimators = []
    predicts = []
    
    for i in range(boot_count):
        new_est = clone(estimator)
        new_est.fit(X_samples[i], y_samples[i])
        predict = new_est.predict(X_test)
        predicts.append(predict)
        estimators.append(new_est)

    predicts = np.array(predicts)
    n = predicts.shape[0]
    y_pred = predicts.sum(axis=0)/n
    
    loss = 0
    bias = 0
    var = 0
    
    m = y_test.shape[0]
    
    predictions = {}
    for i in range(m):
        x = np.array([X_test[i]])
        predictions[i] = {}
        for j in range(n):
            pred = estimators[j].predict(x)
            predictions[i][j] = pred
    
    for predict in predicts:
        temp = 0
        for i in range(m):
            temp += (predict[i] - y_test[i])**2
        temp /= m
        loss += temp    
    loss /= n
    
    for i in range(m):
        yt = y_test[i]
        x = X_test[i]
        temp = 0
        for j in range(boot_count):
            pred =  predictions[i][j]
            temp += pred
        temp /= boot_count
        bias = bias + (yt - temp)**2
    bias /= m
    
    for i in range(m):
        temp = 0
        x = X_test[i]
        for j in range(n):
            pred = predictions[i][j]
            temp2 = 0
            for r in range(n):
                temp2 += predictions[i][r]
            temp2 /= n
            temp = temp + (pred - temp2)**2
        temp /= n
        var += temp
    var /= m
    return y_pred, loss, bias, var

def get_samples(X_test, y_test, boot_count):
    obj_count = X_train.shape[0]
    X_samples = []
    y_samples = []
    for _ in range(boot_count):
        indices = np.random.choice(obj_count, obj_count, True)
        X_sample = []
        y_sample = []
        for index in indices:
            X_sample.append(X_test[index])
            y_sample.append(y_test[index])
        X_samples.append(X_sample)
        y_samples.append(y_sample)
    return X_samples, y_samples

# OPEN TESTS

In [65]:
from sklearn.model_selection import train_test_split

estimator = DTR(max_depth=2)
X_train = np.array([[0, 0], [1, 1], [5, 5], [8, 8], [10, 10]])
y_train = np.array([0, 1, 5, 8, 10])
X_test  = np.array([[4, 4], [6, 6]])
y_test  = np.array([4, 6])

B = 100

y_pred, loss, bias, var = bagging(estimator, X_train, y_train, X_test, y_test, boot_count=B)

# Да я в курсе что очень грубые ограничения, просто пример игрушечный на таком малом количестве данных
assert_array_almost_equal(y_pred, np.array([4, 6]), decimal=0)

assert_almost_equal(loss, 3.7, decimal=0) 
assert_almost_equal(bias, 0.1, decimal=1) 
assert_almost_equal(var,  3.7, decimal=0) 

######################################################
B = 10

from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=123,
                                                    shuffle=True)



tree = DTR(max_depth=7)

y_pred, loss, bias, var = bagging(
        tree, X_train, y_train, X_test, y_test, boot_count=200)

assert_almost_equal(loss, 32, decimal=0) 
assert_almost_equal(bias, 14, decimal=0) 
assert_almost_equal(var,  18, decimal=0) 

print('Well Done!')

[3.65675]
[17.56135542]
Well Done!


# SLIDE (2) RF Classification

Осталось переделать чуток предыдущую задачу в `RandomForest`. 
Но теперь мы наконец попробуем классификацию. (Пока только бинарную)

План
* Также делаем бутстрепные выборки
* Бэггинг теперь будет только по деревьям классификации
* Будем передавать параметр `n_estimators`, `max_depth` и `max_features`

Как выбирать ответ в задаче классификации?
* Для каждого внутреннего дерева решений находим веротности обоих классов для каждого объекта $X_test$:
  * Вызываем `predict_proba` у `DecisionTreeClassifier`
* Усредняем вероятности класса и объекта по деревьям:
  * $P(n_{class}=d, object=x_k) = \frac{1}{B}\sum_{i=1}^{B}P(n_{class}=d, object=x_k, tree=b_i)$
* Для каждого объекта выбираем тот класс, у которого выше вероятность



### Sample 1
#### Input:
```python
X_train = np.array([[0, 0], [4, 4], [5, 5], [10, 10]])
y_train = np.array([0, 0, 1, 1])
X_test  = np.array([[3, 3], [6, 6]])
y_test  = np.array([0, 1])

B = 1000
```
#### Output:
```python
model.predict(X_test) == np.array([0, 1])
```

# TASK

In [206]:
from sklearn.tree import DecisionTreeClassifier as DTC

class MyRFC():
    def __init__(self, n_estimators=10, max_features=None, max_depth=None):
        self.n = n_estimators
        self.estimators_ = []
        for _ in range(n_estimators):
            self.estimators_.append(DTC(max_depth=max_depth, max_features=max_features))
        
    def fit(self, X_train: np.array, y_train: np.array):
        X_samples, y_samples = self._get_samples(X_train, y_train, self.n)
        for i in range(self.n):
            self.estimators_[i].fit(X_samples[i], y_samples[i])
        return self
        
    def predict(self, X_test) -> np.array:
        n = X_test.shape[0]
        probas = []
        for i in range(self.n):
            proba = self.estimators_[i].predict_proba(X_test)
            if proba.shape[1] == 1:
                b = np.zeros((n,2))
                b[:,:-1] = proba
                proba = b
            probas.append(proba)
        me = np.mean(probas, axis=0)
        result = []
        for elem in me:
            if elem[0] >= 0.5:
                result.append(0)
            else:
                result.append(1)
        return result
    
    def predict_proba(self, X_test)-> np.array:
        n = X_test.shape[0]
        probas = []
        for i in range(self.n):
            proba = self.estimators_[i].predict_proba(X_test)
            if proba.shape[1] == 1:
                b = np.zeros((n,2))
                b[:,:-1] = proba
                proba = b
            probas.append(proba)
        me = np.mean(probas, axis=0)
        return me
    
    def _get_samples(self, X_test, y_test, boot_count):
        obj_count = X_train.shape[0]
        X_samples = []
        y_samples = []
        for _ in range(boot_count):
            indices = np.random.choice(obj_count, obj_count, True)
            X_sample = []
            y_sample = []
            for index in indices:
                X_sample.append(X_test[index])
                y_sample.append(y_test[index])
            X_samples.append(X_sample)
            y_samples.append(y_sample)
        return X_samples, y_samples

# OPEN TEST

In [213]:
######################################################
X_train = np.array([[0, 0], [4, 4], [5, 5], [10, 10]])
y_train = np.array([0, 0, 1, 1])
X_test  = np.array([[3, 3], [6, 6], [2, 2]])
y_test  = np.array([0, 1, 0])

B = 1000

y_pred_my = MyRFC(n_estimators = 2, max_depth=3).fit(X_train, y_train).predict(X_test)

assert_array_almost_equal(y_pred_my, np.array([0, 1, 0]))
######################################################
from random import gauss
from sklearn.metrics import accuracy_score

num_samples = 1000
theta = np.linspace(0, 2*np.pi, num_samples)

r1 = 1
r2 = 2

rng = np.random.RandomState(1)

circle = np.hstack([np.cos(theta).reshape((-1, 1)) + (rng.randn(num_samples)[:,np.newaxis] / 8), 
                    np.sin(theta).reshape((-1, 1)) + (rng.randn(num_samples)[:,np.newaxis] / 8)])
lil = r1 * circle
big = r2 * circle
X = np.vstack([lil, big])
y = np.hstack([np.zeros(num_samples), np.ones(num_samples)])

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=123,
                                                    shuffle=True)

y_test = y_test.astype('int')


y_pred_my = MyRFC(n_estimators = 100, 
                  max_depth=1).fit(X_train, y_train).predict(X_test)

assert accuracy_score(y_pred_my, y_test) > 0.85
print('Well Done!')

old proba  [[1.]
 [1.]
 [1.]]
new proba [[1. 0.]
 [1. 0.]
 [1. 0.]]


AssertionError: 
Arrays are not almost equal to 6 decimals

Mismatched elements: 1 / 3 (33.3%)
Max absolute difference: 1
Max relative difference: 1.
 x: array([0, 0, 0])
 y: array([0, 1, 0])

# SLIDE (1) Feature Importance

Просто верните отсортированный массив важности фич, полученные из обученного RandomForest. Фичи нумеруются с 1.

### Sample 1
#### Input:
```python
X = np.array([[0, 0], [0,1], [1, 0], [1, 1]])
y = np.array([0,0,1,1])
```
#### Output:
```python
features= np.array([1, 2])
importance = np.array([0.75, 0.25])

```

# TASK

In [228]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingClassifier

def feature_importance(X, y):
    rf = RandomForestClassifier()
    rf.fit(X, y)
    importance = rf.feature_importances_
    positions = [i+1 for i in range(len(importance))]
    my_dict = dict(zip(importance, positions))
    sorted_importance = sorted(importance, reverse=True)
    features = list(map(lambda x: my_dict[x], sorted_importance))
    return features, sorted_importance

# OPEN TESTS

In [229]:
from sklearn.datasets import make_regression, make_classification

######################################################
X = np.array([[0, 0], [0,1], [1, 0], [1, 1]])
y = np.array([0,0,1,1])

f, i = feature_importance(X, y)

assert_array_equal(f , np.array([1, 2]))
assert i[0] > 0.74
######################################################
X, y = make_classification(n_samples=1000, 
                           n_features=4,
                           n_informative=2,
                           shuffle=False, 
                           random_state=10)
print(feature_importance(X, y))
n = 10
a = np.zeros((n, X.shape[1]))
for i in range(n):
    a[i], _ = feature_importance(X, y) 

assert_array_equal(np.round(a.mean(axis=0)), np.array([2,3,4,1]))

######################################################
print('Well Done!')

([2, 3, 4, 1], [0.4965524985732423, 0.23414282102983167, 0.16000738956112476, 0.10929729083580123])
Well Done!
